### <font color="red">Import packages </font>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### <font color="red">Read the data into csv file</font>

In [2]:
data =pd.read_csv('data/bike_sharing_data/merged_data.csv')

In [3]:
data.head()

,Duration,Start date,End date,Start station number,Member type
0,552,2018-01-01 00:05:06,2018-01-01 00:14:18,31104,Member
1,1282,2018-01-01 00:14:30,2018-01-01 00:35:53,31321,Casual
2,1265,2018-01-01 00:14:53,2018-01-01 00:35:58,31321,Casual
3,578,2018-01-01 00:15:31,2018-01-01 00:25:09,31406,Casual
4,372,2018-01-01 00:18:02,2018-01-01 00:24:15,31618,Member


### <font color="red">Select only the prominant station </font>

In [4]:
data_station_31623=data[data['Start station number']==31623] ##this is the data that we are going to use for EDA

In [5]:
data_station_31623=data_station_31623[["Start date","Member type"]]

In [6]:
data_station_31623.head()

,Start date,Member type
45,2018-01-01 01:13:43,Member
72,2018-01-01 01:49:37,Casual
155,2018-01-01 06:41:10,Member
538,2018-01-01 13:28:19,Member
600,2018-01-01 14:02:33,Member


In [7]:
data_station_31623.tail()

,Start date,Member type
7315815,2020-02-29 22:07:05,Member
7315879,2020-02-29 22:42:54,Member
7315880,2020-02-29 22:43:54,Member
7315894,2020-02-29 22:56:03,Member
7315956,2020-02-29 23:47:02,Member


In [8]:
data_station_31623["Start date"]=pd.to_datetime(data_station_31623["Start date"],format='%Y-%m-%d %H:%M:%S')

In [9]:
data_station_31623['hour of day']=data_station_31623["Start date"].dt.hour

In [10]:
data_station_31623['day of the month']=data_station_31623["Start date"].dt.day

In [11]:
data_station_31623.head()

,Start date,Member type,hour of day,day of the month
45,2018-01-01 01:13:43,Member,1,1
72,2018-01-01 01:49:37,Casual,1,1
155,2018-01-01 06:41:10,Member,6,1
538,2018-01-01 13:28:19,Member,13,1
600,2018-01-01 14:02:33,Member,14,1


In [12]:
data_station_31623['date']=data_station_31623["Start date"].dt.date

In [13]:
count=pd.DataFrame(data_station_31623.groupby(['date','hour of day','Member type'])\
                   ['hour of day'].count().rename('number of users'))

In [14]:
count=count.reset_index()

In [15]:
count.head()

,date,hour of day,Member type,number of users
0,2018-01-01,1,Casual,1
1,2018-01-01,1,Member,1
2,2018-01-01,6,Member,1
3,2018-01-01,13,Member,1
4,2018-01-01,14,Member,1


In [16]:
casual_hourly=count[count['Member type']=="Casual"][['date','hour of day','number of users']]

In [17]:
casual_hourly.head()

,date,hour of day,number of users
0,2018-01-01,1,1
6,2018-01-01,16,1
17,2018-01-02,8,1
36,2018-01-03,7,1
39,2018-01-03,9,1


In [18]:
casual_hourly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4614 entries, 0 to 18452
Data columns (total 3 columns):
date               4614 non-null object
hour of day        4614 non-null int64
number of users    4614 non-null int64
dtypes: int64(2), object(1)
memory usage: 144.2+ KB


In [19]:
casual_hourly.columns

Index(['date', 'hour of day', 'number of users'], dtype='object')

In [20]:
member_hourly=count[count['Member type']=="Member"][['date','hour of day','number of users']]

In [21]:
member_hourly.head()

,date,hour of day,number of users
1,2018-01-01,1,1
2,2018-01-01,6,1
3,2018-01-01,13,1
4,2018-01-01,14,1
5,2018-01-01,15,3


In [22]:
member_hourly=member_hourly.reset_index().drop('index',axis=1)

In [23]:
from datetime import timedelta

In [24]:
member_hourly['date_time']=None
for i in range(len(member_hourly)):
    member_hourly['date_time'][i]=timedelta(hours=int(member_hourly['hour of day'].iloc[i]))+pd.to_datetime(member_hourly['date'].iloc[i])
    

In [25]:
member_hourly=member_hourly[['date_time',"number of users"]]

In [26]:
member_hourly.head()

,date_time,number of users
0,2018-01-01 01:00:00,1
1,2018-01-01 06:00:00,1
2,2018-01-01 13:00:00,1
3,2018-01-01 14:00:00,1
4,2018-01-01 15:00:00,3


In [27]:
casual_hourly=casual_hourly.reset_index().drop('index',axis=1)

In [28]:
casual_hourly.index

RangeIndex(start=0, stop=4614, step=1)

In [29]:
casual_hourly['date_time']=None
for i in range(len(casual_hourly)):
    casual_hourly['date_time'][i]=timedelta(hours=int(casual_hourly['hour of day'].iloc[i]))+pd.to_datetime(casual_hourly['date'].iloc[i])

In [30]:
casual_hourly=casual_hourly[['date_time','number of users']]

In [31]:
casual_hourly.head()

,date_time,number of users
0,2018-01-01 01:00:00,1
1,2018-01-01 16:00:00,1
2,2018-01-02 08:00:00,1
3,2018-01-03 07:00:00,1
4,2018-01-03 09:00:00,1


In [32]:
casual_hourly.shape

(4614, 2)

In [33]:
casual_hourly.to_csv('data/bike_sharing_data/casual_hourly.csv',index=False)

In [34]:
member_hourly.head()

,date_time,number of users
0,2018-01-01 01:00:00,1
1,2018-01-01 06:00:00,1
2,2018-01-01 13:00:00,1
3,2018-01-01 14:00:00,1
4,2018-01-01 15:00:00,3


In [35]:
member_hourly.to_csv('data/bike_sharing_data/member_hourly.csv',index=False)

In [36]:
member_hourly.shape

(13842, 2)

In [37]:
count_by_date=pd.DataFrame(data_station_31623.groupby(['date','Member type'])['hour of day'].count().rename('number of users'))

In [38]:
count_by_date=count_by_date.reset_index()

In [39]:
count_by_date.head()

,date,Member type,number of users
0,2018-01-01,Casual,2
1,2018-01-01,Member,19
2,2018-01-02,Casual,1
3,2018-01-02,Member,125
4,2018-01-03,Casual,6


In [40]:
casual_daily=count_by_date[count_by_date['Member type']=="Casual"][['date','number of users']]

In [41]:
member_daily=count_by_date[count_by_date['Member type']=="Member"][['date','number of users']]

In [42]:
casual_daily.head()

,date,number of users
0,2018-01-01,2
2,2018-01-02,1
4,2018-01-03,6
8,2018-01-06,1
11,2018-01-08,7


In [43]:
casual_daily.shape

(754, 2)

In [44]:
casual_daily.to_csv('data/bike_sharing_data/casual_daily.csv',index=False)

In [45]:
member_daily.head()

,date,number of users
1,2018-01-01,19
3,2018-01-02,125
5,2018-01-03,144
6,2018-01-04,70
7,2018-01-05,84


In [46]:
member_daily.shape

(790, 2)

In [47]:
member_daily.to_csv('data/bike_sharing_data/member_daily.csv',index=False)

### <font color="red">Total(member+calusl) users in daily granularity  </font>

In [48]:
data_station_31623.head()

,Start date,Member type,hour of day,day of the month,date
45,2018-01-01 01:13:43,Member,1,1,2018-01-01
72,2018-01-01 01:49:37,Casual,1,1,2018-01-01
155,2018-01-01 06:41:10,Member,6,1,2018-01-01
538,2018-01-01 13:28:19,Member,13,1,2018-01-01
600,2018-01-01 14:02:33,Member,14,1,2018-01-01


In [49]:
dayly_count=pd.DataFrame(data_station_31623.groupby(['date'])\
                   ['date'].count().rename('number of users'))

In [50]:
dayly_count=dayly_count.reset_index()

In [51]:
dayly_count.head()

,date,number of users
0,2018-01-01,21
1,2018-01-02,126
2,2018-01-03,150
3,2018-01-04,70
4,2018-01-05,84


In [52]:
dayly_count.to_csv('data/bike_sharing_data/daily_count.csv',index=False)